In [1]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, upper, trim, from_json, to_json, json_tuple, get_json_object
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

spark = SparkSession.Builder().appName('from_json(), to_json(), json_tuple() function').getOrCreate()

24/06/18 12:59:21 WARN Utils: Your hostname, codebase resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/18 12:59:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/18 12:59:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
data = [
    ('Rohit', "{'address' : {'city' : 'MTR', 'state' : 'UP'}, 'gender' : 'male'}"),
    ('Ajay', "{'address' : {'city' : 'MTR', 'state' : 'UP'}, 'eye' : 'black'}"),
]

schema = ['name', 'props']

df = spark.createDataFrame(data, schema)
df.show(truncate=False)
df.printSchema()

+-----+-----------------------------------------------------------------+
|name |props                                                            |
+-----+-----------------------------------------------------------------+
|Rohit|{'address' : {'city' : 'MTR', 'state' : 'UP'}, 'gender' : 'male'}|
|Ajay |{'address' : {'city' : 'MTR', 'state' : 'UP'}, 'eye' : 'black'}  |
+-----+-----------------------------------------------------------------+

root
 |-- name: string (nullable = true)
 |-- props: string (nullable = true)



In [12]:
df.select(df.name
          ,get_json_object('props', '$.address.city').alias('city')
          ,get_json_object('props', '$.address.state').alias('state')
          ,get_json_object('props', '$.gender').alias('gender')
          ,get_json_object('props', '$.eye').alias('eye')
         )\
.show()

+-----+----+-----+------+-----+
| name|city|state|gender|  eye|
+-----+----+-----+------+-----+
|Rohit| MTR|   UP|  male| NULL|
| Ajay| MTR|   UP|  NULL|black|
+-----+----+-----+------+-----+



In [21]:
df1 = df.withColumn('city', get_json_object('props', '$.address.city'))\
        .withColumn('state', get_json_object('props', '$.address.state'))\
        .withColumn('gender', get_json_object('props', '$.gender'))\
        .withColumn('eye', get_json_object('props', '$.eye'))
df1.show(truncate=False)
df1.printSchema()

+-----+-----------------------------------------------------------------+----+-----+------+-----+
|name |props                                                            |city|state|gender|eye  |
+-----+-----------------------------------------------------------------+----+-----+------+-----+
|Rohit|{'address' : {'city' : 'MTR', 'state' : 'UP'}, 'gender' : 'male'}|MTR |UP   |male  |NULL |
|Ajay |{'address' : {'city' : 'MTR', 'state' : 'UP'}, 'eye' : 'black'}  |MTR |UP   |NULL  |black|
+-----+-----------------------------------------------------------------+----+-----+------+-----+

root
 |-- name: string (nullable = true)
 |-- props: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- eye: string (nullable = true)



In [22]:
spark.stop()